
... ***CURRENTLY UNDER DEVELOPMENT*** ...


In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

# common
import os
import os.path as op

# pip
import numpy as np
import xarray as xr
import shutil

# DEV: override installed teslakit
import sys
sys.path.insert(0, op.join(os.path.abspath(''), '..', '..','..', '..'))

# teslakit 
from teslakit.alr import ALR_WRP
from teslakit.util.time_operations import xds_reindex_daily, xds_common_dates_daily

from teslakit.plotting.estela import Plot_DWTs_Probs
from teslakit.plotting.wts import Plot_Probs_WT_WT, Plot_Probs_WT_WT_anomaly

from teslakit.io.aux_nc import StoreBugXdset



## Database and Site parameters

In [ ]:
# Teslakit database

site ='ROI'
p_data = r'/Users/albacid/Projects/TeslaKit_projects/sites/' + site + '/'

p_outputs = p_data + 'ClimateChange_SLR_ENSO/'


# TODO: copy alr_w folder 
shutil.copytree(p_data + 'ESTELA/alr_w/', p_outputs + 'ESTELA/alr_w/') 


In [ ]:
# --------------------------------------
# load data and set parameters

KMA_fit = xr.open_dataset(p_data + 'ESTELA/pred_SLP/kma.nc') # Historical bmus. ESTELA + TCs Predictor (daily)
MJO_sim_all = xr.open_dataset(p_data + 'MJO/MJO_sim.nc')        # MJO simulation (daily)
PCs_sim_all = xr.open_dataset(p_outputs + 'SST/CC_SST_PCs_sim_d.nc')     # SST PCs simulation (daily)

# ALR fit parameters
alr_num_clusters = 42
alr_markov_order = 1
alr_seasonality = [2, 4]

# ALR simulation
num_sims = 10 # one simulation for each simulated MJO, SST 


In [ ]:
# --------------------------------------
# Data used to FIT ALR model and preprocess: 

# KMA: bmus (daily) (use sorted_bmus_storms, add 1 to get 1-42 bmus set)
BMUS_fit = xr.Dataset(
    {
        'bmus':(('time',), KMA_fit['sorted_bmus_storms'].values[:] + 1),
    },
    coords = {'time': KMA_fit.time.values[:]}
)




## ESTELA Predictor - Autoregressive Logistic Regression Simulation

In [ ]:
# --------------------------------------
# Prepare Covariates for ALR simulations

# simulation dates
d_sim = xds_common_dates_daily([MJO_sim_all, PCs_sim_all])

# join covariates for all MJO, PCs simulations
l_cov_sims = []
for i in MJO_sim_all.n_sim: 

    # select simulation
    MJO_sim = MJO_sim_all.sel(n_sim=i)
    PCs_sim = PCs_sim_all.sel(n_sim=i)

    # PCs covar 
    cov_PCs = PCs_sim.sel(time = slice(d_sim[0], d_sim[-1]))
    cov_1 = cov_PCs.PC1.values.reshape(-1,1)
    cov_2 = cov_PCs.PC2.values.reshape(-1,1)
    cov_3 = cov_PCs.PC3.values.reshape(-1,1)

    # MJO covars
    cov_MJO = MJO_sim.sel(time = slice(d_sim[0], d_sim[-1]))
    cov_4 = cov_MJO.rmm1.values.reshape(-1,1)
    cov_5 = cov_MJO.rmm2.values.reshape(-1,1)

    # join covars (do not normalize simulation covariates)
    cov_T_sim = np.hstack((cov_1, cov_2, cov_3, cov_4, cov_5))
    cov_sim = xr.Dataset(
        {
            'cov_values': (('time','n_covariates'), cov_T_sim),
            'cov_names': (('n_covariates',), ['PC1','PC2','PC3','MJO1','MJO2']),
        },
        coords = {'time': d_sim}
    )
    l_cov_sims.append(cov_sim)

# use "n_sim" name to join covariates (ALR.Simulate() will recognize it)
cov_sims = xr.concat(l_cov_sims, dim='n_sim')
cov_sims = cov_sims.squeeze()

print(cov_sims)


In [ ]:
# --------------------------------------
# Autoregressive Logistic Regression

# ALR wrapper
ALRW = ALR_WRP(p_outputs + 'ESTELA/alr_w/')

ALRW.LoadModel()
ALRW.LoadBmus_Fit()


In [ ]:
# --------------------------------------
# Autoregressive Logistic Regression - simulate 

# launch simulation
#xds_alr = ALRW.Simulate(num_sims, d_sim, cov_sims)

# Store Daily Weather Types
#DWT_sim = xds_alr.evbmus_sims.to_dataset()
#StoreBugXdset(DWT_sim, p_outputs + 'ESTELA/CC_DWT_sim.nc')

DWT_sim = xr.open_dataset(p_outputs + 'ESTELA/CC_DWT_sim.nc')
print(DWT_sim)


In [ ]:
# show sim report
ALRW.Report_Sim(py_month_ini=6);


In [ ]:
# Plot Simulated DWTs probabilities (with TCs DWTs)

bmus_sim = DWT_sim.isel(n_sim=0).evbmus_sims.values[:]
dbmus_sim = DWT_sim.time.values[:]

Plot_DWTs_Probs(bmus_sim, dbmus_sim, alr_num_clusters);


In [ ]:
# Plot AWTs/DWTs Probabilities 

xds_AWT = xr.open_dataset(p_outputs + 'SST/CC_SST_AWT_sim.nc')

# clusters to plot (no TCs)
n_clusters_AWT = 6
n_clusters_DWT = 42
n_sim = 0  # simulation to plot


# AWT simulated - evbmus_sims -1 
xds_AWT = xr.Dataset(
    {'bmus': (('time',), xds_AWT.evbmus_sims.isel(n_sim=n_sim)-1)},
    coords = {'time': xds_AWT.time.values[:]}
)

# DWT simulated - evbmus_sims -1
xds_DWT = xr.Dataset(
    {'bmus': (('time',), DWT_sim.evbmus_sims.isel(n_sim=n_sim)-1)},
    coords = {'time': DWT_sim.time.values[:]}
)

# reindex AWT to daily dates (year pad to days)
xds_AWT = xds_reindex_daily(xds_AWT)

# get common dates
dc = xds_common_dates_daily([xds_AWT, xds_DWT])
xds_DWT = xds_DWT.sel(time=slice(dc[0], dc[-1]))
xds_AWT = xds_AWT.sel(time=slice(dc[0], dc[-1]))


# Plot AWTs/DWTs sim - simulated
AWT_bmus = xds_AWT.bmus.values[:]
DWT_bmus = xds_DWT.bmus.values[:]

Plot_Probs_WT_WT(
    AWT_bmus, DWT_bmus, n_clusters_AWT, n_clusters_DWT,
    wt_colors=True, ttl = 'DWTs Probabilities by AWTs - Simulation'
);


Plot_Probs_WT_WT_anomaly(
    AWT_bmus, DWT_bmus, n_clusters_AWT, n_clusters_DWT,
    wt_colors=True, ttl = 'DWTs Probabilities by AWTs - anomalies Simulation'
);
